# 0. Load Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!unzip 'drive/MyDrive/kubig_ML/235713_신용카드 사용자 연체 예측 AI 경진대회_data.zip'

Mounted at /content/drive
Archive:  drive/MyDrive/kubig_ML/235713_신용카드 사용자 연체 예측 AI 경진대회_data.zip
   creating: open/
  inflating: open/train.csv          
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           


In [2]:
import pandas as pd
train=pd.read_csv('open/train.csv')
test=pd.read_csv('open/test.csv')

# 1. Preprocessing

In [73]:
train_prep=train.copy()
train_prep.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [74]:
train.columns

Index(['index', 'gender', 'car', 'reality', 'child_num', 'income_total',
       'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email',
       'occyp_type', 'family_size', 'begin_month', 'credit'],
      dtype='object')

## 1.1. 'index','FLAG_MOBIL' 삭제

In [75]:
train_prep=train_prep.drop(columns=['index','FLAG_MOBIL'])

## 1.2. 'gender' , 'car' , 'reality' -> 0,1로 encoding 
* gender M:1, F:0
* car Y:1, N:0
* reality Y:1, N:0

In [76]:
train_prep['gender']=train_prep['gender'].replace(['M','F'],[1,0])
train_prep['car']=train_prep['car'].replace(['Y','N'],[1,0])
train_prep['reality']=train_prep['reality'].replace(['Y','N'],[1,0])

## 1.3. 중복데이터 제거
* 전부 다 중복인 데이터만 제거 : 1642행 제거됨

In [77]:
train_prep=train_prep[~train_prep.duplicated()]

In [78]:
train_prep[train_prep.duplicated()]

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit


In [79]:
train_prep[train_prep.drop(columns='credit').duplicated()]

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
328,1,1,1,1,270000.0,Working,Secondary / secondary special,Married,House / apartment,-14488,-1630,0,1,0,Laborers,3.0,-22.0,2.0
547,1,0,0,1,225000.0,Commercial associate,Secondary / secondary special,Married,With parents,-17868,-1912,0,0,0,Drivers,3.0,-21.0,0.0
992,0,0,1,0,135000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-17579,-1904,1,0,0,Laborers,1.0,-59.0,0.0
1183,0,1,1,0,180000.0,Commercial associate,Higher education,Married,House / apartment,-12953,-306,1,1,0,NaN,2.0,-50.0,2.0
1198,0,0,1,0,135000.0,State servant,Incomplete higher,Single / not married,With parents,-9873,-2086,0,1,1,NaN,1.0,-16.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26397,1,1,1,0,135000.0,Working,Higher education,Married,House / apartment,-9075,-614,1,1,0,Sales staff,2.0,-9.0,2.0
26403,1,0,1,0,270000.0,Working,Secondary / secondary special,Married,House / apartment,-16896,-248,0,0,0,Laborers,2.0,-13.0,1.0
26430,0,0,1,0,112500.0,Working,Incomplete higher,Civil marriage,House / apartment,-9301,-1751,0,0,0,NaN,2.0,-19.0,2.0
26431,0,0,1,0,225000.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-21151,365243,0,0,1,NaN,1.0,-60.0,1.0


In [80]:
train_prep=train_prep[~train_prep.drop(columns='credit').duplicated()]

In [81]:
train_prep[train_prep.drop(columns='credit').duplicated()]

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit


In [82]:
train_prep

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,NaN,2.0,-6.0,1.0
1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0,1.0
2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0,2.0
3,0,0,1,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0,0.0
4,0,1,1,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,0,0,0,Core staff,4.0,-2.0,1.0
26453,0,0,1,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,0,0,0,NaN,2.0,-47.0,2.0
26454,0,1,0,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,0,0,0,Core staff,2.0,-25.0,2.0
26455,1,0,1,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,0,0,0,Laborers,1.0,-59.0,2.0


## 1.4. 'DAYS_EMPLOYED': 365243->0

In [83]:
train_prep['DAYS_EMPLOYED'].loc[train_prep['DAYS_EMPLOYED']>0]=0

## 1.5. 'occyp_type'

In [84]:
tmp=train_prep[['DAYS_EMPLOYED','occyp_type']]

In [85]:
tmp[tmp['occyp_type'].isna()]

,DAYS_EMPLOYED,occyp_type
0,-4709,NaN
8,-1466,NaN
14,0,NaN
18,0,NaN
19,-6031,NaN
...,...,...
26439,0,NaN
26441,0,NaN
26443,0,NaN
26449,-2418,NaN


In [86]:
tmp[(tmp['occyp_type'].isna())&(tmp['DAYS_EMPLOYED']==0)]

,DAYS_EMPLOYED,occyp_type
14,0,NaN
18,0,NaN
21,0,NaN
24,0,NaN
46,0,NaN
...,...,...
26406,0,NaN
26427,0,NaN
26439,0,NaN
26441,0,NaN


In [87]:
#train_prep['occyp_type']=train['occyp_type'].copy()
train_prep['occyp_type'].loc[(train_prep['DAYS_EMPLOYED']==0)&(train['occyp_type'].isna())]='Non_occupied'

In [88]:
train_prep.isna().sum()

gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
work_phone          0
phone               0
email               0
occyp_type       3357
family_size         0
begin_month         0
credit              0
dtype: int64

## 1.5. categorical 변수 label encoding

'income_type','edu_type','family_type','house_type'

In [89]:
train_prep.columns

Index(['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type',
       'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'work_phone', 'phone', 'email', 'occyp_type', 'family_size',
       'begin_month', 'credit'],
      dtype='object')

In [90]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

train_prep['income_type']=label_encoder.fit_transform(train_prep['income_type'])
test['income_type']=label_encoder.transform(test['income_type'])

train_prep['edu_type']=label_encoder.fit_transform(train_prep['edu_type'])
test['edu_type']=label_encoder.transform(test['edu_type'])

train_prep['family_type']=label_encoder.fit_transform(train_prep['family_type'])
test['family_type']=label_encoder.transform(test['family_type'])

train_prep['house_type']=label_encoder.fit_transform(train_prep['house_type'])
test['house_type']=label_encoder.transform(test['house_type'])

TypeError: ignored

In [72]:
train_prep['occyp_type']=label_encoder.fit_transform(train_prep['occyp_type'])
test['occyp_type']=label_encoder.transform(test['occyp_type'])

# 2. Train

In [18]:
X_test=test.drop(columns=['index','FLAG_MOBIL'])

X_test['gender']=X_test['gender'].replace(['M','F'],[1,0])
X_test['car']=X_test['car'].replace(['Y','N'],[1,0])
X_test['reality']=X_test['reality'].replace(['Y','N'],[1,0])

X_test['DAYS_EMPLOYED'].loc[X_test['DAYS_EMPLOYED']>0]=0

X_test['occyp_type'].loc[(X_test['DAYS_EMPLOYED']==0)&(X_test['occyp_type'].isna())]='Non_occupied'

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [329]:
X_train=train_prep.drop(columns='credit')
y_train=train_prep['credit']

In [345]:
from sklearn.ensemble import RandomForestClassifier

model_rf=RandomForestClassifier()
model_rf.fit(X_train,y_train)

y_pred=model_rf.predict(X_test)

In [346]:
y_pred

array([2., 0., 2., ..., 2., 0., 2.])

In [347]:
y_pred_df=pd.DataFrame(y_pred,columns=['credit'],dtype=int)

In [348]:
y_pred_df

,credit
0,2
1,0
2,2
3,2
4,2
...,...
9995,2
9996,1
9997,2
9998,0


In [349]:
y_pred_df=pd.get_dummies(y_pred_df['credit'])

In [350]:
y_pred_df

,0,1,2
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
9995,0,0,1
9996,0,1,0
9997,0,0,1
9998,1,0,0


In [351]:
y_pred_df=pd.concat([test['index'],y_pred_df],axis=1)
y_pred_df.to_csv('base_submission.csv',index = False)